# <b>Document AI features demo 1</b>

The AIServiceVisionClient offers the document <b>text detection</b> feature. This notebook aims to provide overall clarity about the feature to the user in terms of requirements, usage and the output of the API.<br>
<ul>
    <li>The raw output is saved as <code>response_document_demo1.json</code> file. </li>
    <li>Detected text is displayed under <b>Display the lines of text detected</b> section.</li>
    <li>The user can visualize the bounding boxes for the detected text under <b>View output document with bounding boxes</b> section. </li>
</ul>

### Steps to run the notebook:
<details>
    <summary>Notebook session setup</summary>
    <ol>
        <li><font size="2">Installing the OCI Vision SDK</font></li>
        <li><font size="2">Installing other dependencies</font></li>
        <li><font size="2">Setup sample input documents</font></li>
        <li><font size="2">Setup helper .py files</font></li>
        <li><font size="2">Create output folder</font></li>
    </ol>
</details>

<details>
    <summary>Importing the required modules</summary>
</details>

<details>
    <summary>Setting the input variables</summary>
     <font size="2">The user can change the input variables, if necessary. They have been assigned default values.</font>
</details>

<details>
    <summary>Running the main pipeline</summary>
    <font size="2">Run all cells to get the output in the <code>output</code> directory. </font><br>
</details>

### Notebook session setup
<details>
    <summary>Instructions</summary>
    <ul>
        <li><font size="2">The user needs to setup only once.</font></li>
        <li><font size="2">Uncomment the commented cells and run once to setup.</font></li>
        <li><font size="2">Comment back the same cells to avoid running again.</font></li>
    </ul>
</details>

#### Installing the OCI Python SDK

In [ ]:
# !pip install oci==2.57.0

#### Installing other dependencies

In [ ]:
# !pip install matplotlib==3.3.4
# !pip install pandas==1.1.5
# !pip install fitz==0.0.1.dev2
# !pip install pymupdf==1.18.19

#### Setup sample input documents

In [ ]:
# !wget "https://objectstorage.us-ashburn-1.oraclecloud.com/n/axhheqi2ofpb/b/vision-demo-notebooks/o/TextDetectionOnePage.pdf"
# !wget "https://objectstorage.us-ashburn-1.oraclecloud.com/n/axhheqi2ofpb/b/vision-demo-notebooks/o/table.pdf"
# !mkdir data
# !mv TextDetectionOnePage.pdf data
# !mv table.pdf data

#### Setup helper .py files

In [ ]:
# !wget "https://objectstorage.us-ashburn-1.oraclecloud.com/n/axhheqi2ofpb/b/vision-demo-notebooks/o/analyze_document_utils.py"
# !mkdir helper
# !mv analyze_document_utils.py helper

#### Create output folder

In [ ]:
# !mkdir output

### Imports

In [ ]:
import base64
import os
import io
import oci
import json
from IPython.display import IFrame
import requests
from helper.analyze_document_utils import is_url, clean_output

### Set input variables
<details>
<summary><font size="3">input_path</font></summary>
<font size="2">The user can provide the document URL or filepath from the notebook session.</font><br>
</details>

In [ ]:
input_path = "data/TextDetectionOnePage.pdf"

### Authorize user config

In [ ]:
config = oci.config.from_file('~/.oci/config')

### Get input document

In [ ]:
if is_url(input_path):
    file_content = requests.get(input_path).content
    encoded_string = base64.b64encode(file_content)
    input_path = 'data/' + os.path.basename(input_path)
    open(input_path, 'wb').write(file_content)
else:
    with open(input_path, "rb") as document_file:
        encoded_string = base64.b64encode(document_file.read())

### View input document

In [ ]:
if is_url(input_path):
    display(IFrame('data/' + os.path.basename(input_path), width=600, height=500))
else:
    display(IFrame(input_path, width=600, height=500))

### Create AI service vision client and get response object

In [ ]:
ai_service_vision_client = oci.ai_vision.AIServiceVisionClient(config=config)
analyze_document_details = oci.ai_vision.models.AnalyzeDocumentDetails()
inline_document_details = oci.ai_vision.models.InlineDocumentDetails()

text_detection_feature = oci.ai_vision.models.DocumentTextDetectionFeature()
text_detection_feature.generate_searchable_pdf = False
features = [text_detection_feature]
inline_document_details.data = encoded_string.decode('utf-8')
analyze_document_details.document = inline_document_details
analyze_document_details.features = features

res = ai_service_vision_client.analyze_document(analyze_document_details=analyze_document_details)

### Clean and save the API response as json

In [ ]:
res_json = json.loads(repr(res.data))
clean_res = clean_output(res_json)

with open('output/response_document_demo1.json', 'w') as fp:
    json.dump(clean_res, fp)

### Display the lines of text detected

In [ ]:
for i, page in enumerate(clean_res['pages']):
    print('**************** PAGE NO.', i+1, '****************\n')
    for line in page['lines']:
        print(line['text'])
    print('\n')

### View output document with bounding boxes

The user can run the cells below to visualize the bounding boxes over the document. This visualization feature is currently supported for <b>PDF format only.</b>

#### Imports

In [ ]:
from helper.analyze_document_utils import add_text_bounding_boxes_to_pdf
import fitz

#### Add bounding boxes

In [ ]:
doc = fitz.open(input_path)
doc = add_text_bounding_boxes_to_pdf(doc, clean_res)
output_path = 'output/' + 'output_' + os.path.basename(input_path)
doc.save(output_path)

#### Display output document

In [ ]:
display(IFrame(output_path, width=600, height=500))